<a href="https://colab.research.google.com/github/MarohBJoshua22/Emerging-technologies/blob/main/Natuaral_disaster_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1 Loading Dataset


In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/disaster_tweets.csv')

In [2]:
df.shape

(7613, 5)

# 2 Data cleaning

In [3]:
df = df[['text', 'target']] #filtering out all columns except text and target
df = df.drop_duplicates(subset='text') #removing duplicates
df = df.dropna() # removing null values
# utilizing regular expression to remove symbols and urls
import re
def remove_symbols(text):
    return re.sub(r'[^A-Za-z0-9\s]+(?=\s|$)', '', text)
def remove_urls(text):
    url_pattern = r'http[s]?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)

df['text'] = df['text'].apply(remove_symbols)
df['text'] = df['text'].apply(remove_urls)

df.head(5000)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask Canada,1
2,All residents asked to 'shelter in place are b...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
5077,Patriot Survival Guide Do you know how survive...,1
5078,This is the natural and unavoidable consequenc...,0
5079,What Natural Disaster Are You When You Get Angry,0
5080,Is your team ready for a natural disaster a vi...,0


# 3 Data Splitting

In [4]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['target'], test_size=0.2, random_state=42
)

# 4 BERT Tokinization

In [5]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# 5 Encoding Function

In [6]:
import torch

# Function to encode text using BERT
def encode_texts(texts):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
    # max lenth can be 128
    with torch.no_grad():
        outputs = model(**encodings)
    # Use the [CLS] token embedding for classification
    return outputs.last_hidden_state[:, 0, :].numpy()

# 6 Invoking Encode Function

In [7]:

# Encode the training and test texts
train_embeddings = encode_texts(train_texts)
test_embeddings = encode_texts(test_texts)

# 7 Train svm model on BERT Embeddings




In [8]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear')
svm_model.fit(train_embeddings, train_labels)

SVC(kernel='linear')

# 8 Prediction on test-embeddings

In [9]:
test_predictions = svm_model.predict(test_embeddings)

# 9 Model Evaluation

In [10]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy: {accuracy} - {accuracy * 100:.2f}%" )
print(classification_report(test_labels, test_predictions, target_names=['normal-text', 'natuarl-disaster-text']))


Accuracy: 0.7801465689540307 - 78.01%
                       precision    recall  f1-score   support

          normal-text       0.79      0.84      0.82       874
natuarl-disaster-text       0.76      0.69      0.72       627

             accuracy                           0.78      1501
            macro avg       0.78      0.77      0.77      1501
         weighted avg       0.78      0.78      0.78      1501



# 10 Predict function

In [11]:
# Normal predict function to test a single piece of input text
def predict(text):
    # Ensure the input is in a list format for compatibility
    if isinstance(text, str):
        text = [text]

    # Preprocess and encode the input text using BERT
    embeddings = encode_texts(text)

    # Make predictions using the trained SVM model
    predictions = svm_model.predict(embeddings)

    # Return natuaral-disaster-text if the predicted class is 1, else normal-text
    return "natuaral-disaster-text" if predictions[0] == 1 else "normal-text"

# 11 Prediction

In [12]:
import numpy as np
input_text = np.array([
    """
    There is an earth quake
    """
])

prediction = predict(input_text)
print(f"The input text is classified as: {prediction}")

The input text is classified as: natuaral-disaster-text
